# Sistemas Inteligentes

## Procura não informada

## Conteúdos

##### Procura em grafo
* Procura em largura primeiro
* Procura em profundidade primeiro
* Custo uniforme
* Exercícios


## Introdução

Nesta aula vamos ver mais alguns aspetos relacionados com a resolução de problemas de acordo com o Paradigma do Espaço de Estados, usando a implementação disponibilizada pelo módulo `searchPlus.py`, introduzido na aula anterior. Vamos hoje começar por ver como poderemos utilizar os dois algoritmos de procura standard, na sua versão em grafo.

In [2]:
from searchPlus import *

## Grafo com custos heterogéneos
Vamos voltar a usar o grafo abstracto da figura em baixo, com custos heterogéneos.

<img src="figura-ee-T-2.png" alt="Drawing" style="width: 300px;"/>

In [3]:
class ProblemaGrafoCustos(Problem) :
    grafo = {'I':{'A':2,'B':5},
             'A':{'C':2,'D':4,'I':2},
             'B':{'D':1,'F':5,'I':5},
             'C':{},
             'D':{'C':3,'F':2},
             'F':{}}

    def __init__(self,initial = 'I', final = 'F') :
        super().__init__(initial,final)
        
    def actions(self,estado) :
        sucessores = self.grafo[estado].keys()  # métodos keys() devolve a lista das chaves do dicionário
        accoes = list(map(lambda x : "ir de {} para {}".format(estado,x),sucessores))
        return accoes

    def result(self, estado, accao) :
        """Assume-se que uma acção é da forma 'ir de X para Y'
        """
        return accao.split()[-1]
    
    def path_cost(self, c, state1, action, state2):
        return c + self.grafo[state1][state2]

## Profundidade-prim e Largura-prim em Grafo

Na procura em grafo, tanto na profundidade primeiro como na largura primeiro é preciso guardar os estados expandidos num conjunto e também filtrar os estados sucessores, evitando os já expandidos ou aqueles que são terminais dos caminhos na fronteira (ainda não expandidos mas na fronteira).
```python
def graph_search(problem, frontier):
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    If two paths reach a state, only use the first one. [Figure 3.7]"""
    frontier.append(Node(problem.initial))
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None
```

As diferenças são que teremos um conjunto de estados explorados que começa vazio:
```
explored=set()
```
Quando se expande um nó o seu estado é colocado no conjunto de explorados

```
explored.add(node.state)
```

Não se colocam na fronteira todos os nós expandidos, mas apenas aqueles cujo estado não foi ainda explorado, ou que não sejam os estados dos nós da fronteira.

```
frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
```


Notem que a instrução 
```python
child not in frontier
```

só funciona porque dois nós são iguais apenas se os seus atributos **state** o forem, mesmo que os outros atributos não sejam nada iguais!

Eis o algoritmo de procura-primeiro que utiliza o ***graph_search()***
```python
def depth_first_graph_search(problem):
    """Search the deepest nodes in the search tree first."""
    return graph_search(problem, Stack())
```

Notem que os elementos que são inseridos num conjunto têm que ser **hashable**. Temos de ter como estados elementos "hashable" para inserir nos conjuntos e verificar se são membros de conjuntos.

As strings são "hashable" e por isso tudo funciona bem com o grafo abstracto com custos da figura.
Experimentem:

In [4]:
hash("branco")

-2817946740830214814

Como não existe no aima-pyton (módulo `searchPlus.py`) a largura-primeiro em grafo, ei-la aqui.

In [5]:
def breadth_first_graph_search(problem):
    """Search the deepest nodes in the search tree first."""
    return graph_search(problem, FIFOQueue())

Comecemos por criar uma instância da classe do problema e mostrar o grafo e o estado inicial.

In [6]:
p = ProblemaGrafoCustos()
p.initial

'I'

Invoquemos a procura em largura em grafo.

In [7]:
res=breadth_first_graph_search(p)
res.solution()

['ir de I para B', 'ir de B para F']

Notem que o problema dos ciclos com a procura em profundidade neste grafo ficou resolvido!

In [8]:
res=depth_first_graph_search(p)
res.solution()

['ir de I para B', 'ir de B para F']

#### Exercício 1
Faça uma versão da procura em grafo que mostre os estados expandidos ao longo da procura e teste-os para o grafo abstracto, confirmando que os estados nunca são expandidos mais do que uma vez.

In [9]:
## Resolução do exercício 1
def graph_search_echo(problem, frontier):
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    If two paths reach a state, only use the first one. [Figure 3.7]"""
    frontier.append(Node(problem.initial))
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node

        if node.state not in explored:
            print('Expandir', node.state)

        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
        print('Fronteira', frontier)
    return None

In [10]:
def depth_first_graph_search_echo(problem):
    """Search the deepest nodes in the search tree first."""
    return graph_search_echo(problem, Stack())

In [11]:
res=depth_first_graph_search_echo(p)
print(res.solution())

Expandir I
Fronteira [<Node A>, <Node B>]
Expandir B
Fronteira [<Node A>, <Node D>, <Node F>]
['ir de I para B', 'ir de B para F']


## Comparação de procura em árvore vs. em grafo
### Pacman Grave come pastilha
Vamos formular problemas de navegação sujeita às forças da gravidade, numa grelha 2D em que algumas células são obstáculos impassíveis. O estado vai ser um tuplo com as coordenadas (x,y) (x corresponde à coluna e y à linha) e as acções (x+dx,y+dy) correspondem às casas vizinhas, desde que não sejam obstáculos. (dx,dy) corresponde ao deslocamento: (0,1) corresponde ao norte e (-1,-1) a sudoeste.
Notem que as acções para baixo não custam nada, as acções na horizontal custam 1 e as acções para cima custam 2.

In [12]:
class GridProblem(Problem):
    """Encontrar um caminho numa grelha 2D com obstáculos. Os obstáculos são células (x, y)."""

    def __init__(self, initial=(1, 1), goal=(7, 8), obstacles=()):
        self.initial=initial
        self.goal=goal 
        self.obstacles=set(obstacles) - {initial, goal}

    directions = {"N":(0, -1), "W":(-1, 0), "E":(1,  0),"S":(0, +1)}  # ortogonais
    
    costs = {"N":3, "W":1, "E":1,"S":0}  # ortogonais
                  
    def result(self, state, action): 
        "Tanto as acções como os estados são representados por pares (x,y)."
        (x,y) = state
        (dx,dy) = self.directions[action]
        return (x+dx,y+dy) 
    
    def actions(self, state):
        """Podes move-te para uma célula em qualquer das direcções para uma casa 
           que não seja obstáculo."""
        x, y = state
        return [act for act in self.directions.keys() if (x+self.directions[act][0],y+self.directions[act][1]) not in self.obstacles]
    
    def path_cost(self,c,state,action,new):
        return c+self.costs[action]
    
    def display(self, state,d):
        """ print the state please"""
        output=""
        for j in range(d):
            for i in range(d):
                if state ==(i,j):
                    ch = '@'
                elif self.goal==(i,j):
                    ch = "*"
                elif (i,j) in self.obstacles:
                    ch = "#"
                else:
                    ch = "."
                output += ch + " "
            output += "\n"
        print(output)
        
    def display_trace(self,d,plan):
        path = set()
        st = self.initial
        for a in plan[:-1]:
            st = self.result(st,a)
            path.add(st)
            
        """ print the state please"""
        output=""
        for j in range(d):
            for i in range(d):
                if self.initial ==(i,j):
                    ch = '@'
                elif self.goal==(i,j):
                    ch = "*"
                elif (i,j) in self.obstacles:
                    ch = "#"
                elif (i,j) in path:
                    ch = '+'
                else:
                    ch = "."
                output += ch + " "
            output += "\n"
        print(output)

def line(x, y, dx, dy, length):
    """Uma linha de células de comprimento 'length' começando em (x, y) na direcção (dx, dy)."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

def quadro(x, y, length):
    """Uma moldura quadrada de células de comprimento 'length' começando no topo esquerdo (x, y)."""
    return line(x,y,0,1,length) | line(x+length-1,y,0,1,length) | line(x,y,1,0,length) | line(x,y+length-1,1,0,length)

Criemos um mundo cercado 10 x 10, em que o Pacman começa na posição (1,1) e temos uma pastilha em baixo na posição (7,8).

In [13]:
fronteira = quadro(0,0,10)
g = GridProblem(obstacles=fronteira)
g.display(g.initial,10)

# # # # # # # # # # 
# @ . . . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# . . . . . . * . # 
# # # # # # # # # # 



Executemos Profundidade-primeiro em ávore e calculemos o tempo que demora a encontrar a solução:

In [14]:
import timeit
start = timeit.default_timer()
resultado = depth_first_tree_search(g)
print("Solução Prof-prim (árvore) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Podemos ver o rasto do plano')
g.display_trace(10,resultado.solution())

Solução Prof-prim (árvore) com custo 6:
['S', 'S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E']
Time:  0.0006246909979381599
Podemos ver o rasto do plano
# # # # # # # # # # 
# @ . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + + + + + + * . # 
# # # # # # # # # # 



E agora a profundidade primeiro na sua versão em grafo, que por acaso dá a mesma solução do que a versão em árvore:

In [15]:
import timeit
start = timeit.default_timer()
resultado = depth_first_graph_search(g)
print("Solução Prof-prim (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
g.display_trace(10,resultado.solution())

Solução Prof-prim (grafo) com custo 6:
['S', 'S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E']
Time:  0.0004997569994884543
# # # # # # # # # # 
# @ . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + + + + + + * . # 
# # # # # # # # # # 



Vamos comparar a procura em largura primeiro nas suas duas versões: árvore e grafo, para o problema de cima, para o objectivo: (7,8).

In [16]:
import timeit

# em árvore
start = timeit.default_timer()
resultado = breadth_first_tree_search(g)
print("Solução largura-prim (árvore) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
timeTree = stop - start
print('Time: ', timeTree) 

# em Grafo
start = timeit.default_timer()
resultado = breadth_first_graph_search(g)
print("Solução largura-prim (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
timeGraph = stop-start
print('Time: ', timeGraph)

print("A Versão em grafo é",round(timeTree/timeGraph,0),"mais rápida")

Solução largura-prim (árvore) com custo 6:
['E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'S', 'S', 'S', 'S']
Time:  38.592319347997545
Solução largura-prim (grafo) com custo 6:
['E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'S', 'S', 'S', 'S']
Time:  0.0007475820020772517
A Versão em grafo é 51623.0 mais rápida


É notória a vantagem no uso da versão em grafo quando há muitos estados repetidos na árvore.

## Custo Uniforme (grafo)
Se aplicarmos a procura de custo uniforme **teremos sempre a solução óptima**. 

Vamos fazê-lo para o grafo abstracto do início do guião.

In [17]:
resultado = uniform_cost_search(p)
print("solução custo uniforme (grafo) com custo", str(resultado.path_cost)+":")
for x in resultado.solution():
    print(x)

solução custo uniforme (grafo) com custo 8:
ir de I para A
ir de A para D
ir de D para F


In [18]:
start = timeit.default_timer()
resultado = uniform_cost_search(g)
print("Solução custo uniforme (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
timeGraph = stop-start
print('Time: ', timeGraph)
g.display_trace(10,resultado.solution())

Solução custo uniforme (grafo) com custo 6:
['S', 'S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E']
Time:  0.0012955109996255487
# # # # # # # # # # 
# @ . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + . . . . . . . # 
# + + + + + + * . # 
# # # # # # # # # # 



#### Exercício 2.
Calcule os melhores caminhos até F a partir de quaisquer dos estados do grafo. No caso de não haver solução indique-o, como em baixo.

    Menor distância de  I ao objectivo F = 8
    Menor distância de  A ao objectivo F = 6
    Menor distância de  B ao objectivo F = 3
    Sem solução a partir de C
    Menor distância de  D ao objectivo F = 2
    Menor distância de  F ao objectivo F = 0

In [19]:
# Resolução do exercício 2
for estado in p.grafo.keys():
    px = uniform_cost_search(ProblemaGrafoCustos(estado))
    if not px:
        print(f'Sem solução a partir de {estado}')
    else:
        print(f'Menor distancia de {estado} ao objetivo {p.goal} = {px.path_cost}')


Menor distancia de I ao objetivo F = 8
Menor distancia de A ao objetivo F = 6
Menor distancia de B ao objetivo F = 3
Sem solução a partir de C
Menor distancia de D ao objetivo F = 2
Menor distancia de F ao objetivo F = 0


### Por dentro do ***uniform_cost_search()***
O algoritmo de custo uniforme implementado em `searchPlus.py` faz uso de um método genérico ***uniform_cost_search()***, que usa um estrutura de dados ***PriorityQueue*** para armazenar a fronteira. A função que é usada na fila de prioridades é passada como argumento. Neste caso, é o custo que é o critério que estrutura a fila de prioridades, passado como função no segundo argumento, sendo o primeiro a instância do problema.

```python
def uniform_cost_search(problem):
    """[Figure 3.14]"""
    return best_first_graph_search(problem, lambda node: node.path_cost)
```

Olhemos com algum detalhe para o método ***best_first_graph_search***. Como é uma pesquisa em grafo é necessário guardar num conjunto os estados expandidos.

A fronteira é uma **PriorityQueue** que estrutura os seus elementos com base na minimização da função f (no caso do custo uniforme, f devolve o custo do caminho).

Os sucessores de um nó são filtrados de modo a que os estados expandidos nunca sejam expandidos e de modo a que se há um nó na fronteira que seja igual ao do nó sucessor (estados iguais) só o que tem menor custo é que fica na fronteira. Só é adicionado se não tiver sido expandido nem existe na fronteira ou se é melhor do que o que existe na fronteira, que é retirado.

De modo a que não se recalcule sempre a função f, os valores de f são "cached".

Notem que os estados têm de ser comparáveis, tem de existir o método __lt()__ (less than). Os tuplos de inteiros são comparáveis.

```python
def best_first_graph_search(problem, f):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    frontier = PriorityQueue(min, f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                incumbent = frontier[child]
                if f(child) < f(incumbent):
                    del frontier[incumbent]
                    frontier.append(child)
    return None
```

Voltemos ao Pacman vítima da força da gravidade.
Vamos colocar alguns obstáculos no mundo.

In [20]:
l = line(2,2,1,0,6)
c = line(2,3,0,1,4)
g = GridProblem(obstacles=fronteira | l | c,goal=(3,3))
g.display(g.initial,10)

# # # # # # # # # # 
# @ . . . . . . . # 
# . # # # # # # . # 
# . # * . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# # # # # # # # # # 



Executemos a Largura-prim em grafo

In [21]:
# em Grafo
start = timeit.default_timer()
resultado = breadth_first_graph_search(g)
print("Solução largura-prim (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
timeGraph = stop-start
print('Time: ', timeGraph)
g.display_trace(10,resultado.solution())

Solução largura-prim (grafo) com custo 14:
['S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'N', 'N', 'N', 'N']
Time:  0.0012175239971838892
# # # # # # # # # # 
# @ . . . . . . . # 
# + # # # # # # . # 
# + # * . . . . . # 
# + # + . . . . . # 
# + # + . . . . . # 
# + # + . . . . . # 
# + + + . . . . . # 
# . . . . . . . . # 
# # # # # # # # # # 



E agora o custo uniforme, que dá a solução óptima em termos de custo mas que tem mais duas acções do que a devolvida pela largura.

In [22]:
# em Grafo
start = timeit.default_timer()
resultado = uniform_cost_search(g)
print("Solução custo uniforme (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
timeGraph = stop-start
print('Time: ', timeGraph)
g.display_trace(10,resultado.solution())

Solução custo uniforme (grafo) com custo 12:
['E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'W', 'W']
Time:  0.0019220319954911247
# # # # # # # # # # 
# @ + + + + + + + # 
# . # # # # # # + # 
# . # * + + + + + # 
# . # . . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# # # # # # # # # # 



#### Exercício 3.
Desenvolva uma versão em árvore do algoritmo custo uniforme e compare em termos temporais a execução dos dois algoritmos para o mesmo problema do Pacman Grave. Note que terá de na verdade fazer uma versão em árvore do ***best_first_graph_search()***.

In [23]:
## Resolução do exercício 3


#### Exercício 4.
O algoritmo de profundidade primeiro para árvore está resolvido de modo recursivo. Desenvolva uma versão iterativa do algoritmo de profundidade primeiro limitada para árvore, que tem como base o método ***tree_search()***.

```python
def depth_limited_search(problem, limit=50):
    """[Figure 3.17]"""
    def recursive_dls(node, problem, limit):
        if problem.goal_test(node.state):
            return node
        elif limit == 0:
            return 'cutoff'
        else:
            cutoff_occurred = False
            for child in node.expand(problem):
                result = recursive_dls(child, problem, limit - 1)
                if result == 'cutoff':
                    cutoff_occurred = True
                elif result is not None:
                    return result
            return 'cutoff' if cutoff_occurred else None

    # Body of depth_limited_search:
    return recursive_dls(Node(problem.initial), problem, limit)
```


In [24]:
## Resolução do exercício 4


#### Exercício 5.
Com base no exercício 4 desenvolva o algoritmo de aprofundamento progressivo na sua versão em árvore.

In [25]:
## Resolução do exercício 5


#### Exercício 6
Desenvolva a versão iterativa em grafo do algoritmo de aprofundamento progressivo

In [26]:
## Resolução do exercicio 6


#### Exercício 7
Faça novas versões da profundidade-primeiro e largura-primeiro em grafo, usando a ***função best_first_graph_search()*** em que a função passada tem que ver com o comprimento dos nós. Repare que se quer minimizar o comprimento dos nós no caso da largura e maximizar no caso da profundidade.

In [27]:
## Resolução do exercício 7
